In [ ]:
from processBioDBs.utilities import getSequence
from processBioDBs.embedding import *

In [ ]:
getSequenceRepresentation??

In [ ]:
import pandas as pd

In [ ]:
getSequenceRepresentation??

In [ ]:
df = pd.read_csv("/data/projects/processBio/gnomad/gnomad.exomes.r2.1.1.sites.vcf.vep",header=93,delimiter="\t")

In [ ]:
missense = df[df.Consequence == "missense_variant"]

In [ ]:
missense

In [ ]:
missense = missense.assign(info=missense.Extra.apply(lambda s: dict([l.split("=") for l in s.split(";")])))

In [ ]:
missense[missense["info"].apply(lambda i: i["SYMBOL"] == "MTHFR" if "SYMBOL" in i else False)].drop_duplicates(["Protein_position", "Amino_acids"])

# Example for generating an embedding

In [ ]:
def getEmbedding(idx):
    info = dict([i.split("=") for i in missense.loc[idx].Extra.split(";")])

    # get the original (reference) amino acid, the variant, and the location of the variant (1-based so you need to convert to zero based)
    og,var = missense.loc[idx].Amino_acids.split("/")
    loc = int(missense.loc[idx].Protein_position) - 1

    s = getSequence(info["SYMBOL"])
    if s[loc]!=og:
        return []
    sequence = s[:loc] + var + s[loc+1:]

    sequenceRepresentation = getSequenceRepresentation([(idx, sequence)])

    return sequenceRepresentation[0]

In [ ]:
getEmbedding(missense.index[0])

# Embed all missense variants

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
representations = []
for idx in tqdm(missense.index):
    rep = getEmbedding(idx)
    representations.append(rep)